# L01: 競馬予想モデル学習

## モデル比較
1. **オッズなしモデル**: 純粋な予測（レース前情報のみ）
2. **オッズありモデル**: 速報オッズ込み

**実行環境**: ローカル（M4 MacBook Air）

---
## 1. 環境設定

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

RACES_PARQUET = PROJECT_ROOT / 'data' / 'processed' / 'races.parquet'
RESULTS_PARQUET = PROJECT_ROOT / 'data' / 'processed' / 'results.parquet'
MODEL_DIR = PROJECT_ROOT / 'models'
MODEL_DIR.mkdir(exist_ok=True)

print(f"データパス: {RACES_PARQUET}")

In [ ]:
races_df = pd.read_parquet(RACES_PARQUET)
results_df = pd.read_parquet(RESULTS_PARQUET)

print(f"レース数: {len(races_df):,}")
print(f"出走馬データ数: {len(results_df):,}")

---
## 2. データ前処理・分割

- **Train**: 2010〜2024年
- **Test**: 2025年〜

In [ ]:
results_df['race_date'] = pd.to_datetime(results_df['race_date'])
results_df['year'] = results_df['race_date'].dt.year

# 完走馬のみ
results_df = results_df[results_df['is_finished'] == True].copy()

TRAIN_END_YEAR = 2024

train_df = results_df[results_df['year'] <= TRAIN_END_YEAR].copy()
test_df = results_df[results_df['year'] > TRAIN_END_YEAR].copy()

print(f"Train: {len(train_df):,} ({train_df['year'].min()}〜{train_df['year'].max()})")
print(f"Test: {len(test_df):,} ({test_df['year'].min()}〜{test_df['year'].max()})")

---
## 3. 特徴量エンジニアリング

### 追加特徴量
- 前走着順、前走上がり3F、前走からの間隔日数
- 距離別・芝ダート別の過去成績

In [ ]:
def add_previous_race_features(df):
    """
    前走情報を特徴量として追加
    """
    df = df.sort_values(['horse_id', 'race_date']).copy()
    
    # 前走の情報をシフトで取得（同じ馬のみ）
    df['prev_finish'] = df.groupby('horse_id')['finish_position'].shift(1)
    df['prev_last_3f'] = df.groupby('horse_id')['last_3f'].shift(1)
    df['prev_race_date'] = df.groupby('horse_id')['race_date'].shift(1)
    
    # 前走からの間隔日数
    df['days_since_last'] = (df['race_date'] - df['prev_race_date']).dt.days
    
    # 前走がなければ新馬扱い
    df['is_debut'] = df['prev_finish'].isna().astype(int)
    
    return df

print("前走情報を追加中...")
results_with_prev = add_previous_race_features(results_df)
print(f"完了: {len(results_with_prev):,} 行")

In [ ]:
def calculate_historical_features(df, history_df):
    """各出走馬の過去成績から特徴量を計算"""
    history_df = history_df.sort_values('race_date')
    
    # 馬ごとの過去成績
    horse_stats = history_df.groupby('horse_id').agg({
        'finish_position': ['count', 'mean'],
    }).reset_index()
    horse_stats.columns = ['horse_id', 'horse_race_count', 'horse_avg_finish']
    
    win_stats = history_df.groupby('horse_id').apply(
        lambda x: pd.Series({
            'horse_win_rate': (x['finish_position'] == 1).sum() / len(x) if len(x) > 0 else 0,
            'horse_place_rate': (x['finish_position'] <= 3).sum() / len(x) if len(x) > 0 else 0,
            'horse_avg_last_3f': x['last_3f'].mean(),
        })
    ).reset_index()
    horse_stats = horse_stats.merge(win_stats, on='horse_id', how='left')
    
    # 芝・ダート別成績
    for surface_type in ['芝', 'ダート']:
        surface_hist = history_df[history_df['surface'] == surface_type]
        surface_stats = surface_hist.groupby('horse_id').apply(
            lambda x: pd.Series({
                f'horse_{surface_type}_count': len(x),
                f'horse_{surface_type}_place_rate': (x['finish_position'] <= 3).sum() / len(x) if len(x) > 0 else 0,
            })
        ).reset_index()
        horse_stats = horse_stats.merge(surface_stats, on='horse_id', how='left')
    
    # 騎手の成績
    jockey_stats = history_df.groupby('jockey_id').apply(
        lambda x: pd.Series({
            'jockey_race_count': len(x),
            'jockey_win_rate': (x['finish_position'] == 1).sum() / len(x) if len(x) > 0 else 0,
            'jockey_place_rate': (x['finish_position'] <= 3).sum() / len(x) if len(x) > 0 else 0,
        })
    ).reset_index()
    
    # 調教師の成績
    trainer_stats = history_df.groupby('trainer_id').apply(
        lambda x: pd.Series({
            'trainer_race_count': len(x),
            'trainer_win_rate': (x['finish_position'] == 1).sum() / len(x) if len(x) > 0 else 0,
            'trainer_place_rate': (x['finish_position'] <= 3).sum() / len(x) if len(x) > 0 else 0,
        })
    ).reset_index()
    
    result = df.merge(horse_stats, on='horse_id', how='left')
    result = result.merge(jockey_stats, on='jockey_id', how='left')
    result = result.merge(trainer_stats, on='trainer_id', how='left')
    
    return result

print("特徴量計算関数を定義")

In [ ]:
print("特徴量を計算中...")

# 前走情報付きデータで分割
train_with_prev = results_with_prev[results_with_prev['year'] <= TRAIN_END_YEAR].copy()
test_with_prev = results_with_prev[results_with_prev['year'] > TRAIN_END_YEAR].copy()

all_history = results_with_prev[results_with_prev['year'] <= TRAIN_END_YEAR].copy()

train_features = calculate_historical_features(train_with_prev, all_history)
test_features = calculate_historical_features(test_with_prev, all_history)

print(f"Train features: {train_features.shape}")
print(f"Test features: {test_features.shape}")

In [ ]:
def add_final_features(df):
    df = df.copy()
    df['surface_encoded'] = df['surface'].map({'芝': 0, 'ダート': 1}).fillna(-1)
    df['target'] = (df['finish_position'] <= 3).astype(int)
    
    # コース適性（今回のコースでの複勝率）
    df['surface_match'] = np.where(
        df['surface'] == '芝',
        df['horse_芝_place_rate'],
        df['horse_ダート_place_rate']
    )
    
    return df

train_features = add_final_features(train_features)
test_features = add_final_features(test_features)

print("特徴量追加完了")

---
## 4. モデル定義

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# オッズなしモデル（純粋予測）
FEATURE_COLS_NO_ODDS = [
    # レース情報
    'distance', 'surface_encoded', 'level_score',
    'horse_number', 'gate_number', 'impost',
    # 前走情報
    'prev_finish', 'prev_last_3f', 'days_since_last', 'is_debut',
    # 馬の過去成績
    'horse_race_count', 'horse_avg_finish',
    'horse_win_rate', 'horse_place_rate',
    'horse_avg_last_3f', 'surface_match',
    # 騎手の成績
    'jockey_race_count', 'jockey_win_rate', 'jockey_place_rate',
    # 調教師の成績
    'trainer_race_count', 'trainer_win_rate', 'trainer_place_rate',
]

# オッズありモデル
FEATURE_COLS_WITH_ODDS = FEATURE_COLS_NO_ODDS + ['popularity', 'odds']

print("オッズなしモデル特徴量:", len(FEATURE_COLS_NO_ODDS), "個")
print("オッズありモデル特徴量:", len(FEATURE_COLS_WITH_ODDS), "個")

In [ ]:
# データ準備（欠損値を埋める）
for col in FEATURE_COLS_WITH_ODDS:
    if col in train_features.columns:
        train_features[col] = train_features[col].fillna(0)
        test_features[col] = test_features[col].fillna(0)

train_clean = train_features.dropna(subset=['target'])
test_clean = test_features.dropna(subset=['target'])

print(f"Train (clean): {len(train_clean):,}")
print(f"Test (clean): {len(test_clean):,}")

---
## 5. モデル学習・評価

In [ ]:
def train_and_evaluate(train_data, test_data, feature_cols, model_name):
    """モデルの学習と評価"""
    print(f"\n{'='*50}")
    print(f"🎯 {model_name}")
    print(f"{'='*50}")
    
    X_train = train_data[feature_cols]
    y_train = train_data['target']
    X_test = test_data[feature_cols]
    y_test = test_data['target']
    
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'random_state': 42,
    }
    
    train_ds = lgb.Dataset(X_tr, label=y_tr)
    val_ds = lgb.Dataset(X_val, label=y_val, reference=train_ds)
    
    model = lgb.train(
        params,
        train_ds,
        num_boost_round=1000,
        valid_sets=[train_ds, val_ds],
        valid_names=['train', 'valid'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=0)
        ]
    )
    
    y_pred_proba = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred_proba)
    print(f"\n📊 Test AUC: {auc:.4f}")
    
    test_with_pred = test_data.copy()
    test_with_pred['pred_proba'] = y_pred_proba
    
    # 的中率（上位3頭の中に3着内馬がいる割合）
    def calc_hit(group, top_n=3):
        top_preds = group.nlargest(top_n, 'pred_proba')
        return (top_preds['finish_position'] <= 3).any()
    
    race_hit = test_with_pred.groupby('race_id').apply(calc_hit)
    hit_rate = race_hit.mean()
    print(f"🎯 上位3頭予測の的中率: {hit_rate:.1%}")
    
    # 単勝回収率
    def sim_returns(group):
        top = group.nlargest(1, 'pred_proba').iloc[0]
        return top['odds'] * 100 if top['finish_position'] == 1 else 0
    
    returns = test_with_pred.groupby('race_id').apply(sim_returns)
    return_rate = returns.sum() / (len(returns) * 100) * 100
    print(f"💰 単勝回収率: {return_rate:.1f}%")
    
    # 三連複的中率シミュレーション（上位5頭から3頭を選ぶ）
    def calc_sanrenpuku_hit(group):
        top5 = group.nlargest(5, 'pred_proba')
        actual_top3 = group.nsmallest(3, 'finish_position')['horse_id'].values
        pred_top5_ids = top5['horse_id'].values
        # 実際の3着内馬のうち、予測上位5頭に何頭含まれるか
        matches = len(set(actual_top3) & set(pred_top5_ids))
        return matches >= 3  # 3頭全て含まれていれば的中
    
    sanren_hit = test_with_pred.groupby('race_id').apply(calc_sanrenpuku_hit)
    sanren_rate = sanren_hit.mean()
    print(f"🏇 三連複的中率（上位5頭選択）: {sanren_rate:.1%}")
    
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importance()
    }).sort_values('importance', ascending=False)
    
    return {
        'model': model,
        'auc': auc,
        'hit_rate': hit_rate,
        'return_rate': return_rate,
        'sanren_rate': sanren_rate,
        'importance': importance,
        'predictions': test_with_pred
    }

In [ ]:
result_no_odds = train_and_evaluate(
    train_clean, test_clean,
    FEATURE_COLS_NO_ODDS,
    "オッズなしモデル（純粋予測）"
)

In [ ]:
result_with_odds = train_and_evaluate(
    train_clean, test_clean,
    FEATURE_COLS_WITH_ODDS,
    "オッズありモデル（速報オッズ込み）"
)

---
## 6. モデル比較

In [ ]:
print("\n" + "="*60)
print("📊 モデル比較サマリー")
print("="*60)

comparison = pd.DataFrame({
    '指標': ['AUC', '上位3頭的中率', '三連複的中率（5頭選択）', '単勝回収率'],
    'オッズなし': [
        f"{result_no_odds['auc']:.4f}",
        f"{result_no_odds['hit_rate']:.1%}",
        f"{result_no_odds['sanren_rate']:.1%}",
        f"{result_no_odds['return_rate']:.1f}%"
    ],
    'オッズあり': [
        f"{result_with_odds['auc']:.4f}",
        f"{result_with_odds['hit_rate']:.1%}",
        f"{result_with_odds['sanren_rate']:.1%}",
        f"{result_with_odds['return_rate']:.1f}%"
    ]
})

print(comparison.to_string(index=False))

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, (name, result) in zip(axes, [
    ('オッズなしモデル', result_no_odds),
    ('オッズありモデル', result_with_odds)
]):
    imp = result['importance'].head(12)
    ax.barh(imp['feature'], imp['importance'])
    ax.set_xlabel('Importance')
    ax.set_title(f'{name}\nTop 12 Features')
    ax.invert_yaxis()

plt.tight_layout()
plt.show()

---
## 7. モデル保存

In [ ]:
import pickle

with open(MODEL_DIR / 'model_no_odds.pkl', 'wb') as f:
    pickle.dump(result_no_odds['model'], f)

with open(MODEL_DIR / 'model_with_odds.pkl', 'wb') as f:
    pickle.dump(result_with_odds['model'], f)

config = {
    'feature_cols_no_odds': FEATURE_COLS_NO_ODDS,
    'feature_cols_with_odds': FEATURE_COLS_WITH_ODDS,
    'train_end_year': TRAIN_END_YEAR,
}

with open(MODEL_DIR / 'model_config.pkl', 'wb') as f:
    pickle.dump(config, f)

print("✅ モデル保存完了")
print(f"  - {MODEL_DIR / 'model_no_odds.pkl'}")
print(f"  - {MODEL_DIR / 'model_with_odds.pkl'}")
print(f"  - {MODEL_DIR / 'model_config.pkl'}")